In [1]:
import os
os.environ["JAVA_HOME"] = "/mnt/extproj/projekte/textmining/jdk/openlogic-openjdk-11.0.22+7-linux-x64"
import glob

from collections import Counter

In [2]:
from collections import defaultdict
from pyspark.sql import SparkSession

from pyspark.sql.functions import *
from pyspark.sql.types import *

from pyspark.sql import Window


In [3]:
if not "spark" in globals() or spark is None:
    print("Creating builder")
    spark = SparkSession.builder.\
            config("spark.executor.memory", "70g").\
            config("spark.driver.memory", "50g").\
            config("spark.memory.offHeap.enabled",True).\
            config("spark.memory.offHeap.size","16g").\
            config("spark.sql.shuffle.partitions", 300).\
            appName('mirexplore').getOrCreate()
    
#spark.conf.set("spark.executor.memory", "70g")
#spark.conf.set("spark.driver.memory", "50g")
#spark.conf.set("spark.memory.offHeap.enabled",True)
#spark.conf.set("spark.memory.offHeap.size","16g")  
#spark.conf.set("spark.sql.shuffle.partitions", 300)

Creating builder


/mnt/extproj/projekte/bartelt/software/miniconda3/envs/mirexplore/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [4]:
df_int = spark.read.parquet("/mnt/extproj/projekte/textmining/mx_feb24/mx_mirna_gene_consensus_parquet/")
df_int = df_int.withColumn("doc_id", explode("evidence_documents"))

In [5]:
relDocIDs = set([x["doc_id"] for x in df_int.select("doc_id").collect()])

In [6]:
len([x for x in relDocIDs if x.startswith("PMC")])

55331

In [7]:
def read_sentence_file(inputpath, relevantDocIDs):
    db_entries = []
    ignoredConcepts = Counter()


    for infile in glob.glob("{}/*.sent".format(inputpath)):
        with open(infile, "r") as fin:
            
            for iline, line in enumerate(fin):
                aline = line.split("\t")

                sentID = aline[0]
                docID = sentID.split(".")[0]
                
                if not docID in relevantDocIDs:
                    continue
                
                sentence = aline[1]
                
                db_entries.append( (docID, sentID, sentence) )
                
    return db_entries


In [8]:
docSentences = read_sentence_file("/mnt/extproj/projekte/textmining/pubmed_feb24/", relDocIDs)
print(len(docSentences))


555266


In [9]:
docSentencesPMC = read_sentence_file("/mnt/extproj/projekte/textmining/pmc_feb24/oa_comm/", relDocIDs)
print(len(docSentencesPMC))

12507407


In [10]:
print(len(docSentences) + len(docSentencesPMC))

13062673


In [11]:
df = spark.createDataFrame(docSentences+docSentencesPMC,
                           ["doc_id", "sent_id", "sentence"])

In [12]:
df.show()

+--------+------------+--------------------+
|  doc_id|     sent_id|            sentence|
+--------+------------+--------------------+
|10760272|10760272.1.1|MIR16, a putative...|
|10760272|10760272.2.1|We have identifie...|
|10760272|10760272.2.2|MIR16 shares stro...|
|10760272|10760272.2.3|It interacts with...|
|10760272|10760272.2.4|Analysis of delet...|
|10760272|10760272.2.5|MIR16 is an integ...|
|10760272|10760272.2.6|MIR16 represents ...|
|10760272|10760272.2.7|The putative enzy...|
|12576545|12576545.1.1|GDE1/MIR16 is a g...|
|12576545|12576545.2.1|Previously we ide...|
|12576545|12576545.2.2|Here we show that...|
|12576545|12576545.2.3|Analysis of subst...|
|12576545|12576545.2.4|The GPI-PDE activ...|
|12576545|12576545.2.5|Membrane topology...|
|12576545|12576545.2.6|Our results sugge...|
|12812784|12812784.1.1|Temporal regulati...|
|12812784|12812784.2.1|lin-4 and let-7 a...|
|12812784|12812784.2.2|In Caenorhabditis...|
|12812784|12812784.2.3|To identify Droso...|
|12812784|

In [13]:
df.write.parquet("/mnt/extproj/projekte/textmining/mx_feb24/mx_document_sentences", mode="overwrite")